In [4]:
print(os.listdir())

['.ipynb_checkpoints', 'dukest1resurvey2009.3d', 'dukest1resurvey2009.gltf', 'dukest1resurvey2009.gltf.import', 'dukest1resurvey2009.js', 'dukest1resurvey2009.tscn', 'DukeStResurvey-drawnup-p3.jpg', 'DukeStResurvey-drawnup-p3.jpg.import', 'parse3ddmp.py', 'parse3D_work.ipynb', '__pycache__']


In [1]:
from parse3ddmp import DMP3d
import os

dump3dexe = '"C:\\Program Files (x86)\\Survex\\dump3d.exe"'
fname = "dukest1resurvey2009.3d"
fjson = "dukest1resurvey2009.json"
fname = "cusseypot\\Cussey_Pot_Master.3d"
fjson = "cusseypot\\Cussey_Pot_Master.json"

dstream = os.popen("%s -d %s" % (dump3dexe, fname))
lines = dstream.readlines()
dmp3d = DMP3d(lines)

"""
    self.nodepmap = { }   # { p: ([names], [flagsconsolidated] }  
    self.nodes = [ ]      # [ (p, name, [flags]) ]   (not strictly necessary, except to get entrance name before flagconsolidation)
    self.lines = [ ]      # [ (prevp, p, name, style, [flags], date) ]
    self.xsects = [ [ ] ] # [ [ (name, lrud) ] ]
    self.nmapnodes = { }  # { name: p } (derived from nodepmap, needed for xsects)
""";

In [2]:
from parse3ddmp import P3
import json

leglines = [line  for line in dmp3d.lines  if line[0] != line[1] and "SURFACE" not in line[4]]
xsects = [xsect  for xsect in dmp3d.xsects  if len(xsect) >= 2]

assert not dmp3d.cs, "We need to convert from absolute coords in this case"

# points as triples
stationpoints = set()
stationpoints.update(line[0]  for line in leglines)
stationpoints.update(line[1]  for line in leglines)
stationpoints = list(stationpoints)
stationpoints.sort(key=lambda X: (X[2], X[0], X[1]))

# centralize the sketch
def rangerat(ps, i, maxfac):
    return (1-maxfac)*min(p[i]  for p in ps) + maxfac*max(p[i]  for p in ps)
svxp0 = P3(rangerat(stationpoints, 0, 0.5), rangerat(stationpoints, 1, 0.5), rangerat(stationpoints, 2, 1.0))

stationpointsdict = dict((p, i)  for i, p in enumerate(stationpoints))
stationpointscoords = sum((tuple(stationpoint - svxp0)  for stationpoint in stationpoints), ())
stationpointsnames = [ dmp3d.nodepmap[stationpoint][0][0]  for stationpoint in stationpoints ]
legsconnections = sum(((stationpointsdict[legline[0]], stationpointsdict[legline[1]])  for legline in leglines), ())
legsconnections = sum(((stationpointsdict[legline[0]], stationpointsdict[legline[1]])  for legline in leglines), ())
legsstyles = [ legline[3]  for legline in leglines ]

xsectgps = [ ]
for xsectseq in xsects:
    xsectpoints = [dmp3d.nmapnodes[xs[0]]  for xs in xsectseq]
    xsectindexes = [ stationpointsdict[xsectpoint]  for xsectpoint in xsectpoints ]
    xsectrightvecs = [ ]
    for i in range(len(xsectpoints)):
        pback = xsectpoints[max(0, i-1)]
        p = xsectpoints[i]
        pfore = xsectpoints[min(len(xsectpoints)-1, i+1)]
        vback = P3.ZNorm(P3(p.x - pback.x, p.y - pback.y, 0.0))
        vfore = P3.ZNorm(P3(pfore.x - p.x, pfore.y - p.y, 0.0))
        xsectplanevec = P3.ZNorm(vback + vfore)
        xsectrightvecs.append(xsectplanevec[1])
        xsectrightvecs.append(-xsectplanevec[0])
    xsectrightvecs
    xsectlruds = sum((xs[1]  for xs in xsectseq), ())
    xsectlruds
    xsectgps.append({ "xsectindexes":xsectindexes, "xsectrightvecs":xsectrightvecs, "xsectlruds":xsectlruds })
xsectgps

jrec = { "stationpointscoords": stationpointscoords,
         "stationpointsnames":  stationpointsnames, 
         "legsconnections":     legsconnections,
         "legsstyles":          legsstyles, 
         "xsectgps":            xsectgps
       }

open(fjson, "w").write(json.dumps(jrec))

172586

In [3]:
stationpoints

[P3(421572.78, 376489.51, 178.44),
 P3(421573.56, 376489.56, 178.65),
 P3(421572.1, 376490.38, 178.81),
 P3(421572.14, 376492.16, 179.02),
 P3(421572.59, 376489.71, 179.23),
 P3(421572.85, 376489.95, 179.28),
 P3(421572.9, 376490.63, 179.28),
 P3(421572.32, 376489.75, 179.31),
 P3(421567.05, 376497.42, 179.34),
 P3(421567.84, 376497.25, 179.4),
 P3(421572.46, 376489.74, 179.4),
 P3(421572.59, 376490.13, 179.49),
 P3(421572.24, 376490.15, 179.51),
 P3(421567.16, 376497.91, 179.71),
 P3(421570.76, 376495.8, 179.73),
 P3(421567.9, 376498.3, 179.76),
 P3(421570.74, 376492.97, 179.78),
 P3(421572.52, 376492.21, 179.83),
 P3(421571.57, 376491.65, 179.84),
 P3(421571.17, 376492.15, 179.87),
 P3(421572.01, 376492.39, 179.87),
 P3(421572.24, 376492.18, 179.88),
 P3(421571.44, 376492.41, 179.91),
 P3(421567.88, 376497.22, 179.94),
 P3(421571.46, 376493.03, 180.0),
 P3(421567.89, 376498.31, 180.04),
 P3(421568.26, 376499.08, 180.06),
 P3(421567.59, 376497.49, 180.08),
 P3(421567.14, 376497.57, 18

In [ ]:
E 0:00:01.176   audio_device_init: WASAPI: Initialize failed with error 0xffffffff80004005.
  <C++ Error>   Condition "hr != ((HRESULT)0x00000000)" is true. Returned: ERR_CANT_OPEN
  <C++ Source>  drivers/wasapi/audio_driver_wasapi.cpp:299 @ audio_device_init()

E 0:00:01.677   init_library:  does not have a library for the current platform.
  <C++ Error>   Condition "lib_path.length() == 0" is true.
  <C++ Source>  modules/gdnative/nativescript/nativescript.cpp:1483 @ init_library()
